# User Study on Interpretability - Tutorial

#### Thank you for participating in our study! 
 
The study is structured as follows:
1. **Tutorial**: Overview of background concepts that will be used in the rest of the study.
2. **Setup Description**: Presentation of the dataset used, descriptive statistics, etc. 
3. **Main Component**: Using the interpretability tool to answer questions about an ML model.
4. **Follow-Up**: Questionnaire and interview. 

Please keep this tutorial open and handy while you complete the study. **You're welcome (and encouraged!) to refer back to it at any point.**


## Concepts covered in this tutorial


Weight of Evidence:
* Definition
* Interpretation
* Binary vs. Multiclass
* Grouping Features
* Sequential Explanations
* Example
    
---

## Weight Of Evidence

<img align="center" width="500" height="500" src="./images_tutorial/woe_balance.png">

### Setup 

The study will focus on machine learning classifiers, so we will introduce the Weight of Evidence using a (made up) classification example.

Suppose we are trying to predict the variable $Y$, where:

* $Y=1$: the patient has the flu
* $Y=0$: the patient does not have the flu

based on binary indicators $X$ of symptoms (e.g., $X_{\text{cough}}=1$ if the person has a cough, and $0$ otherwise). 

Throughout this section we will use "being sick" and "having the flu" interchangeably for $Y=1$, and "being healthy" and "not having the flu" for $Y=0$.

Disclaimer: any specific values (probabilities, etc.) provided here are entirely fabricated for illustration purposes.

---

### Definition

The ***Weight of Evidence*** (WoE for short), is a concept used to quantify feature importance, and it attempts to answer the question:
> "does the *evidence* speak in favor or against a certain *hypothesis*?"

In this study we are interested in 'explaining' predictions of ML classifiers, so the 'evidence' will be some of the input features (e.g., symptoms), the 'hypothesis' will usually be the model's prediction (e.g., $Y=$'sick') and the question we seek to answer is:

> "**according to the model**, how much does the input speak in favor of a certain prediction"

As an example, suppose we want to understand how having a cough (i.e, $X_{\text{cough}}=1$) affects the probability (according to the classifier) of having the flu (i.e., $Y=1$). The Weight-of-Evidence of $X_{\text{cough}}=1$ towards $Y=1$ is: 

$$  \text{woe}(Y=1 : X_{\text{cough}}=1) \quad \overset{\text{def. 1}}{=}  \quad \log \frac{\text{Odds}(Y=1 \mid X_{\text{cough}}=1)}{\text{Odds}(Y=1)} $$

Conveniently for those who find probabilities more intuitive than odds, the WoE has a (mathematically equivalent) alternative definition:

$$  \text{woe}(Y=1 : X_{\text{cough}}=1) \quad \overset{\text{def. 2}}{=}  \quad  \log \frac{P(X_{\text{cough}}=1 \mid Y=1)}{P(X_{\text{cough}}=1 \mid Y=0)}$$

which uses probabilities instead of odds, and is 'reverted' in the sense that it quantifies the change in probability of the evidence $X$ instead of the outcome $Y$.

These equivalent definitions lead to two alternative interpretations. For our example above, suppose 

$$\text{woe}(Y=1 : X_{\text{cough}}=1)= \log 2 \approx 0.30 $$

for some patient. This can be interpreted as:

* **Odds Interpretation:** "The posterior odds of *having the flu* double after taking into account the cough (compared to the prior odds of having it)"  


* **Likelihood Interpretation:** "A person is twice as likely to *have a cough* if they have the flu,  compared to when they are healthy"

To simplify things, from now on we will simply show woe scores as real numbers $\in (-\infty, \infty)$ (e.g., 0.30 for the example above) but will provide guidelines on how to convert/interpret them below.

---

### Interpreting WoE Scores

Regargless of which of the two *formal* interpretations we use, *colloquially*, using the language of the Weight of Evidence literature, we would say:
* $\text{woe}(Y=1 : X_{\text{cough}}=1) > 0 \qquad \Longrightarrow \qquad$ the presence of cough ***speaks in favor*** of this patient having the flu ($Y=1$)

* $\text{woe}(Y=1 : X_{\text{cough}}=1) = 0  \qquad \Longrightarrow \qquad$ the presence of cough doesn't speak for nor against this patient having the flu ($Y=1$)

* $\text{woe}(Y=1 : X_{\text{cough}}=1) < 0  \qquad \Longrightarrow \qquad$ the presence of cough ***speaks against*** this patient having the flu ($Y=1$)

But what does the *magnitude* of the WoE tell us about the *strength* of the evidence? This table provides rule-of-thumb guidelines:

| Weight of Evidence Score | Odds Ratio (Interp. 1) $=$ Probability Ratio (Interp. 2)  | Strength of Evidence |
| -------------   |:-------------:         |  -----:|
| $0$ to  $1.15$  | $1$ to $3$          | Not worth mentioning |
| $1.15$ to $2.3$ | $3$ to $10$   | Substantial |
| $2.3$ to $4.61$ | $10$ to $100$ | Strong |
| $>4.61$         | $> 100$       | Decisive |

**Note:** This same table, with negative values, can be used to quantify the evidence *against* the hypothesis.

Let's see a few concrete examples for our medical diagosis setting:

$$ \begin{align}
\text{woe}(Y=1 : X_{\text{fever}}=1) = 3  & \qquad \Longrightarrow \qquad \text{having a fever provides} \textbf{ strong } \text{evidence that the patient has the flu} \\
\text{woe}(Y=1 : X_{\text{headache}}=1) = 0.7  & \qquad \Longrightarrow \qquad \text{having a headache provides some, but not much, evidence of a flu} \\
\text{woe}(Y=1 : X_{\text{cough}}=0) = -2  & \qquad \Longrightarrow \qquad \textbf{not having } \text{a cough provides substantial evidence } \textbf{against} \text{ having a flu} \\
\text{woe}(Y=1 : X_{\text{nausea}}=0) = 0  & \qquad \Longrightarrow \qquad \textbf{not having } \text{nausea provides no evidence for nor against having a flu} \\
\end{align}$$


### WoE Scores are Additive

When considering multiple features (e.g., symptoms) simultaneously, individual WoE scores **can be added** to obtain a combined total WoE score.

For example, if $X_{\text{cough}}$ and $X_{\text{fever}}$ are independent, then:

$$\text{woe}(Y=1 : X_{\text{cough}}=1 , X_{\text{fever}}=0 ) \;\; = \;\;\;\; \text{woe}(Y=1 : X_{\text{cough}} =1)  \;\;+ \;\; \text{woe}(Y=1 : X_{\text{fever}} =0)$$

For ML classifiers, this will allow us to break down a prediction (related to the left-hand side) into individual feature contributions (right-hand side).


***



### Checkpoint Questions

Suppose $X_{\text{cough}},X_{\text{nausea}},X_{\text{chills}}$ are independent. If for a given patient, we have:
* Fact 1: $\qquad \text{woe}(Y=1 : X_{\text{cough}}=0 )= -1$
* Fact 2: $\qquad \text{woe}(Y=1 : X_{\text{nausea}}=1 ) = 0$
* Fact 3: $\qquad \text{woe}(Y=1 : X_{\text{chills}} =1)=3.3$

**Q1:** How would you interpret these facts? You can use either of the two **interpretations** (odds or probabilities).

Answer:

In [ ]:
# Fact 1 can be interpreted as: 
# Fact 2 can be interpreted as: 
# Fact 3 can be interpreted as: 

**Q2:** What is the **total** weight-of-evidence of these symptoms toward this patient having the flu? How would you interpret this?

Answer:

In [1]:
# The total weight of evidence is _____. This can be interpreted as: 


**Please check in with the researchers when you get to this point.**


****

### Binary vs. Multiclass

Since $Y$ is binary in our example so far, there's only two possible hypotheses: 
* $Y=1$: the patient has the flu (let's say this is the 'primary hypothesis') or
* $Y=0$: the patient does not have the flu (the 'alternative hypothesis').  

Since there are only two hypotheses, evidence *against* one of these is evidence *in favor* of the other.

But what if this were multi-class classification instead? E.g., suppose the model must instead predict one of $K$ possible conditions, and that for a given patient the model predicts $Y=$'flu', which we take as the primary hypothesis. The alternative hypothesis $h'$ could be:
* All the other possible diseases, e.g., $h': \;\; Y \in \{\text{'cold'}, \text{'strep'}, \text{'allergies'},\dots\}$
* Another specific disease, e.g., $h': \;\; Y=\text{'cold'}$
* Any other subset of diseases, e.g., 'viral' or 'bacterial'

Each of these might shed light on different aspects of the prediction. We will always indicate clearly what $h$ and $h'$ are for a specific instance.

For our running example, taking $Y=\text{'flu'}$ and $Y=\text{'any other disease except flu'}$ as primary/alternate hypothesis, a possible WoE decomposition would be:
<img align="center" width="900" height="900" src="./images_tutorial/Simple_WoE_Diagram.001.png">

---

### WoE of Individual Features and Feature Groups

<!--- (<img align="center" width="500" height="500" src="./images_tutorial/dendrogram.png">) --->

In the plot above, we showed the WoE score of each feature. But when the number of features is large, and there is a meaningful way to group them, it is often convenient to show WoE scores **aggregated by group** of features.

For our running example, a sensible grouping of the six symptoms would be:  
* 'respiratory' (cough, dispnea)
*  overall 'body' feeling (aches, weakness)
* 'temperature' (chills, fever). 

In that case, we could instead display:

<img align="center" width="900" height="900" src="./images_tutorial/Simple_WoE_Diagram.002.png">

which might let us quickly realize that the most decisive factors supporting this prediction are respiratory.

---

### Sequential Explanations

So far we have shown 'one-shot' explanations: the WoE of the predicted class against all other classes. But when there's multiple classes, it is sometimes useful to **break down** the explanation into various 'steps'. 

For our diagnosis example, suppose the model predicts 'flu'. It might be illustrative to understand:
1. What evidence points to *viral diseases* ('flu', 'avian flu', etc.) instead of *bacterial* ones ('strep', etc).
2. What evidence singles out common 'flu' over other viral diseases.

For this purpose, we can use the Weight of Evidence iteratively with increasingly refined hypotheses: 

**First**, we produce an explanation for why the model would predict 'viral' instead of 'bacterial':

<img align="center" width="900" height="900" src="./images_tutorial/Simple_WoE_Diagram.003.png">

Note that the total WoE does indeed favor 'viral', which is expected since the model's prediction ('flu') falls within this category.

**Next**, we produce an explanation for why the model predicted 'flu' an not any other label in the 'viral' class:

<img align="center" width="900" height="900" src="./images_tutorial/Simple_WoE_Diagram.004.png">

Of course, we could group the diseases in some other way (e.g., severe vs. mild, contagious vs. non-contagious, etc), leading to different WoE sequences.


---

### Prior and Posterior Probabilities

In order to understand how the WoE influences a model's prediction, we need one more key component: **the prior class probabilities**.

Intuitively, if the prior (i.e., marginal) probability for a certain class is low (e.g., because the training data was very unbalanced), then the *evidence* in favor of it would have to quite strong in order for the model to predict it (over more frequently occurring labels).

Fortunately, the WoE has another property that formalizes this very intuition, and it's given by the identity:

> Posterior log-odds = Prior log-odds + Weight of Evidence

For example,

$$ \underbrace{\log \frac{P(Y=\text{'flu'} \mid X_{\text{cough}}, X_{\text{fever}}, \dots)}{P(Y\neq\text{'flu'} \mid X_{\text{cough}}, X_{\text{fever}}, \dots)}}_{\text{Posterior log-odds (i.e. odds of predictions)}} \quad = \quad \underbrace{\log\frac{P(Y=\text{'flu'})}{P(Y\neq\text{'flu'})}}_{\text{Prior log-odds}} \quad + \quad\underbrace{\log \frac{P(X_{\text{cough}} \mid Y=\text{'flu'})}{P(X_{\text{cough}} \mid Y\neq\text{'flu'})} \quad+ \quad\log \frac{P(X_{\text{fever}} \mid Y=\text{'flu'})}{P(X_{\text{fever}} \mid Y\neq\text{'flu'})} \quad + \cdots}_{\textit{Weight of evidence scores}} $$

Therefore, adding up all the WoE scores plus the log priors, we obtain **posterior log-odds**, which are directly related the model's predictions.

<!---
While log ratios of probabilities might be less intuitive than raw probabilities, they are necessary to use the simple additive identity above. Also, we can still interpret them using the same guidelines and rule of thumb as those for the WoE shown before.
--->

**A Word of Caution**: Note that it could be that *all* classes have negative prior log-odds. This just means all classes have prior probability $<0.5$, which is common in multi-class classification. **What matters here is the relative order of these prior odds.**


Consider this example:

<img align="center" width="500" height="500" src="./images_tutorial/priors.png?q=3">

From this plot, we can say:
* Although both have negative prior log-odds, the ones for $Y=\text{'strep'}$ are much lower (more negative) that those of $Y=\text{'allergies'}$ (i.e., 'strep' is significantly less likely a priori than 'allergies').
* Even moderately strong evidence favoring $Y=\text{'allergies'}$ might be enough to tip the prediction towards that class.
* The evidence **would have to be much stronger** for the model to predict $Y=\text{'strep'}$ because it has to overcome very unfavorable prior odds.



---

### A Real Example

Consider the following example. A patient has fever, no cough, and no other symptoms, i.e.,
$$ X_{\text{fever}} =1, X_{\text{cough}}=0, \text{  and  } X_i=0 \text{ for every other symptom}$$

and the machine learning classifier predicts $Y = \text{'flu'}$. Again, we assume the symptoms are independent.

The following is an actual plot produced by our WoE-Explainer tool for this example:

<img align="center" width="700" height="700" src="./images_tutorial/example_expl.png?modified=81">

Note that:
* The features are now displayed vertically (instead of horizontally), **but the meaning and interpretation remains the same as before**.
* As before, blue and red bars denote positive and negative weight-of-evidence, respectively.
* The shade of the bars encodes the degree significance of WoE according to the table above.

---

### Checkpoint Questions
Based on the WoE explanation above, please answer the following questions:

**Q3:** According to this classifier, does having a fever increase or decrease the odds of having a flu for this person? What about a having a cough?

Answer: 

In [2]:
# For this classifier, [having/not having] a fever [increases/decreases] the odds of having a flu.
# For this classifier, [having/not having] a cough [increases/decreases] the odds of having a flu.

**Q4:** Do you think the prediction would change if now $X_{\text{fever}}=0$? Why/why not?


Answer:

In [ ]:
# If now X_fever=0, the prediction _______, because _____ 

**Please check in with the researchers when you get to this point.**


--- 
## Key Takeaways:

The Weight-of-Evidence (WoE) ... 
1. Helps us answer: "which features speak in favor/against the prediction of the model?"
2. It is the log of odd/probability ratio, **additive** over features
3. Interpretation: $ \qquad
  \text{woe}(Y=c : X) \quad \left. \begin{cases} > 0 \\ <0 \end{cases}\right\} 
  \quad \Longrightarrow \quad  X \quad \text{'speaks'} \quad
  \left.
  \begin{cases}
    \text{in favor of} \\
    \text{against}
  \end{cases}  
  \right\} 
  \text{ class } c
$
4. It can be computed for **individual** features or **groups** of features
5. It can be **one-shot** (predicted class vs 'rest') or **sequential** (compare to subsets of classes)

---

**Please let the researchers know once you're done.**